In [ ]:
helpers.get_star_info(picid=str(vary_series.index[1]), cursor=catalog_cursor)

In [ ]:
frame_idx = 0

for frame_idx in tqdm_notebook(range(len(fits_files)), total=len(fits_files)):

    target_row = target_table.iloc[frame_idx]
    ref_row = ref_table.iloc[frame_idx]

    date_obs = target_row.name

    frame_data = fits.getdata(os.path.join(base_dir, target_row.file))

    show_stamps=False

    # Add in aperture sums for 4 and 6 pixel apertures
    for size in [4, 6]:

        stamp_slice = helpers.get_stamp_slice(target_row.x, target_row.y, stamp_size=(size, size), ignore_superpixel=True)
        stamp = frame_data[stamp_slice]

        ref_slice = helpers.get_stamp_slice(ref_row.x, ref_row.y, stamp_size=(size, size), ignore_superpixel=True)
        ref_stamp = frame_data[ref_slice]    

        # Record the sum values
        target_sum = (stamp - target_row.background).sum()
        target_table.loc[date_obs, 'sum_{}'.format(size)] = target_sum
        
        ref_sum = (ref_stamp - ref_row.background).sum()
        ref_table.loc[date_obs, 'sum_{}'.format(size)] = ref_sum        

        if show_stamps:

            # Show stamps
            fig = plot.show_stamps([stamp, ref_stamp], stamp_size=size, show_residual=True)
            fig.suptitle('{}                        {}'.format(picid, ref_id), y=.9, fontsize=14)
            display(fig)

            # Show normalized stamps
            fig = plot.show_stamps([stamp / stamp.sum(), ref_stamp / ref_stamp.sum()], stamp_size=size, show_residual=True)
            fig.suptitle('{}                        {}'.format(picid, ref_id), y=.9, fontsize=14)
            display(fig)    

            print('*' * 80)

In [ ]:
plt.figure(figsize=(12, 9))

target_table.resample('5T').mean().sum_4.plot(c='r')
ref_table.resample('5T').mean().sum_4.plot(c='r', ls='--')

target_table.resample('5T').mean().sum_6.plot(c='b')
ref_table.resample('5T').mean().sum_6.plot(c='b', ls='--')

target_table.resample('5T').mean().flux_auto.plot(c='g')
ref_table.resample('5T').mean().flux_auto.plot(c='g', ls='--')

plt.legend()